# Práctica 2 segunda parte

---

## Perfilamiento de MaxFlowAeiu

El objetivo consiste en reimplementar nuestro método numérico realizado en la parte dos de la práctica 1 con niveles de BLAS, cómputo en paralelo (CPU/GPU), con compilación a C (por ejemplo vía cython, rcpp) o julia guiándose del perfilamiento de memoria, uso de procesador o tiempo de ejecución de su paquete. 

Usando una máquina de AWS con las siguientes características:

`m4.16xlarge`

`AMI ubuntu 20.04 - ami-042e8287309f5df03`

`100 GB de almacenamiento`

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   39 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              2
Core(s) per socket:              4
Socket(s):                       1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           142
Model name:                      Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz
Stepping:                        12
CPU MHz:                         2303.998
BogoMIPS:                        4607.99
Virtualization:                  VT-x
Hypervisor vendor:               Microsoft
Virtualization type:             full
L1d cache:                       128 KiB
L1i cache:                       128 KiB
L2 cache:                        1 MiB
L3 cache:                        8 MiB
Vulnerability Itlb multihit:  

In [2]:
%%bash
uname -ar #r for kernel, a for all

Linux LAPTOP-O3MKHNJA 5.10.16.3-microsoft-standard-WSL2 #1 SMP Fri Apr 2 22:23:49 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


## Objetivo

---

In [3]:
%pip install MaxFlowAeiu

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from MaxFlowAeiu.MaxFlowAeiu import MaxFlowAeiu
import pandas as pd
import networkx as nx
import random
from pytest import approx
from IPython.display import HTML, display

In [2]:
url_d = "https://raw.githubusercontent.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/main/BD/d.csv"
d = pd.read_csv(url_d,header=None)
arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))

The maximum flow in this network is 1480.0


## Perfilamiento: medición de tiempo en Python y IPython

---

Lo más natural que podemos pensar en medir es el tiempo de ejecución de nuestros códigos. Python y IPython tienen herramientas para este propósito.

### Módulo: time

---

In [3]:
import time

In [4]:
start_time = time.time()

url_d = "https://raw.githubusercontent.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/main/BD/d.csv"
d = pd.read_csv(url_d,header=None)
arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))
end_time = time.time()

The maximum flow in this network is 1480.0


In [5]:
secs = end_time-start_time
print("El cálculo del flujo máximo usando MaxFlowAeiu tomó",secs,"segundos" )

El cálculo del flujo máximo usando MaxFlowAeiu tomó 0.268449068069458 segundos


Probamos que el paquete resuelve correctamente el problema con la función `maximum_flow` del paquete `Scipy`

Veamos el mismo ejercicio pero usando `Scipy`

In [6]:
# Scipy
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow


Para poder usar la función de flujo máximo de `Scipy`, es necesario tener la matriz en formato _sparse_, una vez representada de esta manera, es sencillo encontrar el valor del fliujo máximo.

In [7]:
# Generamos el arreglo final de tipo "numpy array" para su uso con la función de Scipy
arreglo = d.to_numpy()
arreglo
arreglo2=arreglo.astype(int)
graph = csr_matrix(arreglo2)
maximum_flow(graph, 0, 43).flow_value

1480

Con el paquete `MaxFlowAeiu`:

In [8]:
arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
mfaeiu=MF.ford_fulkerson()

Comparando resultados:

In [9]:
flow_val=maximum_flow(graph, 0, 43).flow_value
print(flow_val == mfaeiu)

True


### Comando de magic: `%time`

--- 

Este comando nos regresa las mediciones siguientes:

**CPU times** que contiene:

* _user_ : mide la cantidad de tiempo de los statements que la CPU gastó para funciones que no están relacionadas con el kernel del sistema.

* _sys_ : mide la cantidad de tiempo de los statements que la CPU gastó en funciones a nivel de kernel del sistema.

* _total_ : suma entre el user y sys para todos todos los cores.

**Wall time:** mide el wall clock o elapsed time que se refiere al tiempo desde que inicia la ejecución de los statements hasta su finalización.

**Out:** resultado.


In [10]:
%time 
arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))

CPU times: total: 0 ns
Wall time: 0 ns
The maximum flow in this network is 1480.0


* Tardó 14 milisegundos para funciones que no están relacionadas con el kernel del sistema.
* Tardó 0 milisegundos para funciones a nivel kernel del sistema.
* Tentiendo un total de 14 milisegundos unidades de tiempo.
* Tardó 26.9 milisegundos desde el inicio hasta el fin de la ejecución de la función.

----- ACTUALIZAR CUANDO SE CORRA EN LA INSTANCIA DE AWS ------

**Timeit**

Se ejecuta desde la línea de comandos, con el comando de magic `%timeit` o realizando `import`.

A continuación se mide el tiempo de ejecución para la función del flujo máximo.

In [11]:
import timeit

In [17]:
start = timeit.timeit()

arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))

end = timeit.timeit()

The maximum flow in this network is 1480.0


In [19]:
secs = end-start
print("El cálculo del flujo máximo usando MaxFlowAeiu tomó",secs,"segundos" )

El cálculo del flujo máximo usando MaxFlowAeiu tomó 0.005848400003742427 segundos


### **cProfile**

---

cProfile está en la standard-library de Python como built-in. Se utiliza con la implementación CPython de Python para medir el tiempo de ejecución de cada función en el programa. Se ejecuta desde la línea de comandos, con un comando de magic o realizando import.

El output de cProfile muestra:

* El tiempo **total** de ejecución, el cual incluye el tiempo del bloque de código que estamos midiendo y el overhead al usar cProfile. Por esta razón se tiene un mayor tiempo de ejecución que con las mediciones de tiempo anteriores.

* La columna **ncalls** que como el nombre indica, muestra el número de veces que se llamó a cada función. En este caso las funciones lambda y math.exp son las que se llaman un mayor número de veces: $n=106$ veces.

* La columna **tottime** muestra el tiempo que tardaron estas funciones en ejecutarse (sin llamar a otras funciones).

* La columna **percall** es el cociente entre tottime y ncalls.

* La columna **cumtime** contiene el tiempo gastado en la función y en las demás que llama. 

* La columna de **percall** es un cociente entre la columna cumtime y el conteo del número de veces que se llamaron a funciones primitivas o también nombradas built in functions.

* La última columna indica información del _script_ de _python_ que se está ejecutando, la función y la línea en la que se encuentra dentro del código. 

In [20]:
import cProfile

In [21]:
cprof = cProfile.Profile()
cprof.enable()

arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))

cprof.disable()
cprof.print_stats(sort='ncalls')

The maximum flow in this network is 1480.0
         553 function calls in 0.004 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      158    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
      158    0.000    0.000    0.000    0.000 {method 'pop' of 'list' objects}
       65    0.000    0.000    0.000    0.000 {built-in method builtins.min}
       12    0.000    0.000    0.000    0.000 {built-in method builtins.next}
       11    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        8    0.000    0.000    0.000    0.000 dis.py:449(findlinestarts)
        8    0.000    0.000    0.000    0.000 compilerop.py:174(extra_flags)
        8    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
        6    0.002    0.000    0.002    0.000 MaxFlowAeiu.py:27(busq_anchura)
        4    0.000    0.000    0.000    0.000 contextlib.py:86(__init__)
        4    0.000    0.000    0.000 

* Observamos que la función que fue llamada más veces fue `append` al igual que `pop` con un total de 158 veces cada una, aunque de las que hicimos nosotros la más llamada es la de búsqueda de anchura con 6 llamadas y la que menos fue llamada fue `ford_fulkerson` la cual se encarga de obtener el flujo máximo de la red.
* El tiempo total que tardaron estas funciones en correr de forma independiente fueron 0.016 segundos para todas las funciones.


## Perfilamiento: medición de uso de memoria en Python

---

Si bien las computadoras de hoy en día tienen una gran cantidad de RAM es importante que las aplicaciones no utilicen la totalidad pues en ese caso se tendrá una penalización en el performance de la aplicación al utilizar _virtual memory_.

**memory_profiler**

---

Se ejecuta desde la línea de comandos, con un comando de magic o realizando _import_. Al instalar `memory_profiler` se incluyen dos comandos de magic: **%memit** y **%mprun**. Este último (**%mprun**) es similar a `line_profiler` al analizar línea por línea el uso de memoria.

En el caso de import regresa una lista de valores de uso de memoria en MiB medidas cada cierto interval (argumento de **memory_usage**). En lo siguiente se pide que se regrese el máximo uso de memoria de la lista.

In [27]:
from memory_profiler import memory_usage

In [28]:
%load_ext memory_profiler

%memit devuelve el pico de memoria usada en una celda de un jupyter notebook y utiliza las mismas ideas para reportar las mediciones que %timeit:

In [29]:
%memit 

peak memory: 215.61 MiB, increment: 0.55 MiB


In [30]:
arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
%memit -c MF.ford_fulkerson()

peak memory: 217.70 MiB, increment: 2.01 MiB


## Conclusiones

El siguiente paso es mejorar el tiempo de nuestro codigo, considerando que hicimos varios tipos de perfilamiento para ser evaluados. Observamos que sí hubo una mejora en el tiempo de ejecusion, pero que en contraste, se afecto de manera negativa en el conusmo de memoria, aunque consideramos que esto es mínimo.

## Referencias
[1] [Capítulo 5.2 del Libro de Optimización](https://itam-ds.github.io/analisis-numerico-computo-cientifico/5.optimizacion_de_codigo/5.2/Herramientas_de_lenguajes_y_del_SO_para_perfilamiento_e_implementaciones_de_BLAS.html])